In [60]:
# Connection Strings - json

# Connection Strings - SQL database (where we are writing to)

# Translate json file into objects

# Fstrings to write dynamic SQL code into tables

# Close connection strings

In [61]:
# Import Depencies

import matplotlib.pyplot as plt
import polars as pl
import pandas as pd
import numpy as np
import requests
import time
import json
from pprint import pprint
from apikey import apikey

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [62]:
import requests
from requests.structures import CaseInsensitiveDict

#get latitude, longitude, and radius for starting point - New York City is [40.712776 N, -74.005974 W], Germany is [51.1657 N, 10.4515 E]  Germany country code is "de"
#London is [51.507351, -0.127758] Brisbane is [-27.469770, 153.025131]
#I can find London, but US cities are not coming up


default = [0, 0]
germany = [51.1657, 10.4515]
london = [51.507351, -0.127758]
brisbane = [-27.469770, 153.025131]
custom = [0,0] #use your own coordinates - probably not necessary


#set the location to search
location_search = germany


lat = "-27.469770"
lon = "153.025131"


radius_meters = 5000
country_code = ''
limit = 5
offset = 0 #default is zero

location_filter = f'circle:{location_search[1]},{location_search[0]},{radius_meters}'

if country_code != "":
    country_code = country_code + f"|countrycode:{country_code}"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"

#resp = requests.get(url, headers=headers)

#print(resp.status_code)

In [63]:
# List of categories
# catering, commercial.food_and_drinks, accomodations, entertainment, leisure, parking, wheelchair
category_string = ""

activity = True
commercial = True
commercial_catering = False
commercial_supermarket = True
accomodations = False
entertainment = True
leisure = False
parking = False
wheelchair = False


In [64]:
# dictionary of each category that will be captured
category_dict = {'activity' : activity, 'commercial' : commercial, 'commerical.catering' : commercial_catering, 'accomodation' : accomodations, \
                        'entertainment' : entertainment, 'leisure': leisure, 'parking' : parking, 'wheelchair.yes' : wheelchair}

string_of_categories = ""

for k,v in category_dict.items():
    if v:
        string_of_categories = string_of_categories + k + ','

#remove last comma
string_of_categories = string_of_categories[:-1]

# conditions, organic.only, vegetarian.only, wheelchair.yes
url2 = f"https://api.geoapify.com/v2/places?categories={string_of_categories}&filter={location_filter}&limit={limit}&offset={offset}&apiKey={apikey}"

# atms in radius, 

# contact information for the restaurants, opening/closing hours, web address, phone number

In [65]:
# get API dictionary
resp = requests.get(url2, headers=headers)

print(resp.status_code)

json_info = resp.json()
pprint(json_info)

200
{'features': [{'geometry': {'coordinates': [10.456235098490145,
                                            51.20527724579283],
                            'type': 'Point'},
               'properties': {'address_line1': 'Thüringentherme',
                              'address_line2': 'Lindenbühl 10, 99974 '
                                               'Mühlhausen, Germany',
                              'categories': ['building',
                                             'entertainment',
                                             'entertainment.water_park'],
                              'city': 'Mühlhausen',
                              'country': 'Germany',
                              'country_code': 'de',
                              'county': 'Unstrut-Hainich-Kreis',
                              'datasource': {'attribution': '© OpenStreetMap '
                                                            'contributors',
                                             '

In [66]:
pd.json_normalize(json_info["features"]).columns.values

array(['type', 'properties.name', 'properties.country',
       'properties.country_code', 'properties.state', 'properties.county',
       'properties.city', 'properties.postcode', 'properties.street',
       'properties.housenumber', 'properties.lon', 'properties.lat',
       'properties.formatted', 'properties.address_line1',
       'properties.address_line2', 'properties.categories',
       'properties.details', 'properties.datasource.sourcename',
       'properties.datasource.attribution',
       'properties.datasource.license', 'properties.datasource.url',
       'properties.datasource.raw.name',
       'properties.datasource.raw.sauna',
       'properties.datasource.raw.osm_id',
       'properties.datasource.raw.leisure',
       'properties.datasource.raw.website',
       'properties.datasource.raw.building',
       'properties.datasource.raw.operator',
       'properties.datasource.raw.osm_type',
       'properties.datasource.raw.addr:city',
       'properties.datasource.raw.addr

In [67]:
features_pd = pd.json_normalize(json_info["features"])

In [68]:
properties_df = features_pd[["properties.address_line1", "properties.address_line2", "properties.place_id"]]
properties_df

,properties.address_line1,properties.address_line2,properties.place_id
0,Thüringentherme,"Lindenbühl 10, 99974 Mühlhausen, Germany",51e39a96a597e924405986a55886469a4940f00102f901...
1,Kulturhistorisches Museum,"Kristanplatz 7, 99974 Mühlhausen, Germany",51c4751e9e3dea2440599cb067306a9a4940f00102f901...
2,Trend - Gepäck,"Johannisstraße 9, 99974 Mühlhausen, Germany",519fc37c7901e624405941865d80d89a4940f00103f901...
3,Big Lebowski,"Bei der Marienkirche 15, 99974 Mühlhausen, Ger...",51dc95027745e92440591643f707db9a4940f00103f901...
4,C. Strecker,"Bei der Marienkirche 12, 99974 Mühlhausen, Ger...",519c887e6dfde8244059a0eabab7d89a4940f00103f901...


In [69]:
categories_df = features_pd[["properties.categories", "properties.place_id"]]
categories_df

,properties.categories,properties.place_id
0,"[building, entertainment, entertainment.water_...",51e39a96a597e924405986a55886469a4940f00102f901...
1,"[building, building.tourism, entertainment, en...",51c4751e9e3dea2440599cb067306a9a4940f00102f901...
2,"[commercial, commercial.clothing, commercial.c...",519fc37c7901e624405941865d80d89a4940f00103f901...
3,"[commercial, commercial.clothing, commercial.c...",51dc95027745e92440591643f707db9a4940f00103f901...
4,"[commercial, commercial.books, wheelchair, whe...",519c887e6dfde8244059a0eabab7d89a4940f00103f901...


In [70]:
no_duplicates_df = categories_df.drop_duplicates(subset = "properties.place_id", inplace = True)
no_duplicates_df